## Contributor: Yuxin Zhang
## This file has the following goals:
### 1. align the time measurement of the (sectional order data  and congestion) with that of the weather measurement data
### 2. Merge [sectional order data and weather data],  [sectional congestion data and weather data] on (date) & (hour) 
### 3. drop the unnecessary variables
### 4. Add dummy to section_id, hour_of_day, day_of_week

## order_data

In [30]:
order_data = pd.read_csv("selected_order_count.csv")
order_data.head()


section_idx_large  year_month_day_hour  day_of_week  order_in_section  \
0   874172101ffffff  2017-05-01 16:00:00            0                 1   
1   874172101ffffff  2017-08-05 17:00:00            5                 1   
2   874172101ffffff  2017-10-29 12:00:00            6                 1   
3   874172953ffffff  2017-05-20 16:00:00            5                 1   
4   874172953ffffff  2017-07-13 21:00:00            3                 1   

         date  hour  
0  2017-05-01    16  
1  2017-08-05    17  
2  2017-10-29    12  
3  2017-05-20    16  
4  2017-07-13    21

In [31]:
order_data.dtypes

section_idx_large      object
year_month_day_hour    object
day_of_week             int64
order_in_section        int64
date                   object
hour                    int64
dtype: object

#### load the weather data

In [36]:
weather_data = pd.read_csv("wdata.csv")
weather_data.head()

date   time  speed  precipitation  tempurature
0  2017-06-01  00:00      7            0.1           29
1  2017-06-01  01:00      7            0.1           29
2  2017-06-01  02:00      7            0.0           28
3  2017-06-01  03:00      7            0.0           28
4  2017-06-01  04:00      7            0.0           29

### consistent the time data format

In [37]:
hour = []
time = weather_data["time"]
for i in time:
    if str(i)[0] == '0':
        hour.append(int(str(i)[1]))
    else:
        hour.append(int(str(i)[:2]))
weather_data["time"] = hour

In [38]:
weather_data.head()

date  time  speed  precipitation  tempurature
0  2017-06-01     0      7            0.1           29
1  2017-06-01     1      7            0.1           29
2  2017-06-01     2      7            0.0           28
3  2017-06-01     3      7            0.0           28
4  2017-06-01     4      7            0.0           29

In [39]:
weather_data = weather_data.rename(columns = {"time":"hour"})

In [40]:
weather_data.head()

date  hour  speed  precipitation  tempurature
0  2017-06-01     0      7            0.1           29
1  2017-06-01     1      7            0.1           29
2  2017-06-01     2      7            0.0           28
3  2017-06-01     3      7            0.0           28
4  2017-06-01     4      7            0.0           29

### merge the order with the weather dataset

In [41]:
merged_order = pd.merge(order_data, weather_data,  how='left', left_on=['date','hour'], right_on = ['date','hour'])

In [42]:
merged_order = merged_order.dropna()

In [43]:
len(merged_order)

17555

In [44]:
merged_order = merged_order.drop(columns = ["year_month_day_hour","date"])

In [45]:
merged_order.head()

section_idx_large  day_of_week  order_in_section  hour  speed  \
44   874172954ffffff            4                 1     7    7.0   
45   874172954ffffff            4                 1    18    7.0   
46   874172954ffffff            4                 1    19    7.0   
47   874172954ffffff            6                 1     0    6.0   
48   874172954ffffff            6                 1    16    5.0   

    precipitation  tempurature  
44            0.0         33.0  
45            0.0         40.0  
46            0.0         38.0  
47            0.0         35.0  
48            0.0         44.0

In [46]:
merged_order.to_csv("merged_order_non_dummy.csv", index = False)

### adding dummy to the section_large_id, day_of_weak, and the hour of day

In [1]:
import pandas as pd
merged_order = pd.read_csv("merged_order_non_dummy.csv")

In [3]:
merged_order.tail()

section_idx_large  day_of_week  order_in_section  hour  speed  \
17550   874172120ffffff            4                28    20    6.0   
17551   874172120ffffff            4                37    21    7.0   
17552   874172120ffffff            4                13    22    7.0   
17553   874172120ffffff            4                 5    23    7.0   
17554   874172120ffffff            4                17    23    7.0   

       precipitation  tempurature  
17550            0.5         34.0  
17551            0.1         32.0  
17552            0.1         32.0  
17553            0.0         31.0  
17554            0.0         31.0

In [49]:
merged_order= pd.get_dummies(merged_order, columns=['day_of_week'])

In [51]:
merged_order = pd.get_dummies(merged_order, columns = ["section_idx_large"])

In [52]:
merged_order = pd.get_dummies(merged_order, columns = ["hour"])

In [53]:
merged_order.head()

order_in_section  speed  precipitation  tempurature  day_of_week_0  \
0                 1    7.0            0.0         33.0              0   
1                 1    7.0            0.0         40.0              0   
2                 1    7.0            0.0         38.0              0   
3                 1    6.0            0.0         35.0              0   
4                 1    5.0            0.0         44.0              0   

   day_of_week_1  day_of_week_2  day_of_week_3  day_of_week_4  day_of_week_5  \
0              0              0              0              1              0   
1              0              0              0              1              0   
2              0              0              0              1              0   
3              0              0              0              0              0   
4              0              0              0              0              0   

    ...     hour_14  hour_15  hour_16  hour_17  hour_18  hour_19  hour_20  \
0   ...           0        0        0        0        0        0        0   
1   ...           0        0        0        0        1        0        0   
2   ...           0        0        0        0        0        1        0   
3   ...           0        0        0        0        0        0        0   
4   ...           0        0        1        0        0        0        0   

   hour_21  hour_22  hour_23  
0        0        0        0  
1        0        0        0  
2        0        0        0  
3        0        0        0  
4        0        0        0  

[5 rows x 128 columns]

In [54]:
merged_order.to_csv("merged_order_dummy.csv", index = False)

## congestion dataset

In [60]:
weather_data.dtypes

date              object
hour               int64
wind_speed         int64
precipitation    float64
sensible_temp      int64
dtype: object

In [61]:
weather_data.head()

date  hour  wind_speed  precipitation  sensible_temp
0  2017-05-01     0           7            0.0             21
1  2017-05-01     1           6            0.0             22
2  2017-05-01     2           4            0.0             23
3  2017-05-01     3           5            0.0             24
4  2017-05-01     4           6            0.0             25

In [74]:
congestion_data = pd.read_csv("selected_congestion.csv")
congestion_data.head()

small_section_idx           batch_time    tti  speed  day_of_week  \
0   88417281d1fffff  2018-01-01 00:00:00  1.242   96.5            0   
1   88417281d1fffff  2018-01-01 01:00:00  1.280   91.1            0   
2   88417281d1fffff  2018-01-01 02:00:00  1.358   86.9            0   
3   88417281d1fffff  2018-01-01 03:00:00  1.418   86.9            0   
4   88417281d1fffff  2018-01-01 04:00:00  1.299   88.9            0   

         date  hour  
0  2018-01-01     0  
1  2018-01-01     1  
2  2018-01-01     2  
3  2018-01-01     3  
4  2018-01-01     4

### converting to match the time_frame

In [75]:
import datetime
date = congestion_data["date"]
date_new = []
for i in date:
    date_new.append((datetime.datetime.strptime(i, '%Y-%m-%d').date() - datetime.timedelta(days=365)).strftime("%Y-%m-%d"))

In [76]:
congestion_data["date"] = date_new

In [77]:
congestion_data.head()

small_section_idx           batch_time    tti  speed  day_of_week  \
0   88417281d1fffff  2018-01-01 00:00:00  1.242   96.5            0   
1   88417281d1fffff  2018-01-01 01:00:00  1.280   91.1            0   
2   88417281d1fffff  2018-01-01 02:00:00  1.358   86.9            0   
3   88417281d1fffff  2018-01-01 03:00:00  1.418   86.9            0   
4   88417281d1fffff  2018-01-01 04:00:00  1.299   88.9            0   

         date  hour  
0  2017-01-01     0  
1  2017-01-01     1  
2  2017-01-01     2  
3  2017-01-01     3  
4  2017-01-01     4

In [78]:
congestion_data = congestion_data.drop(columns = ["batch_time"])


In [79]:
congestion_data.head()

small_section_idx    tti  speed  day_of_week        date  hour
0   88417281d1fffff  1.242   96.5            0  2017-01-01     0
1   88417281d1fffff  1.280   91.1            0  2017-01-01     1
2   88417281d1fffff  1.358   86.9            0  2017-01-01     2
3   88417281d1fffff  1.418   86.9            0  2017-01-01     3
4   88417281d1fffff  1.299   88.9            0  2017-01-01     4

In [80]:
merged_congestion = pd.merge(congestion_data, weather_data,  how='right', left_on=['date','hour'], right_on = ['date','hour'])

In [81]:
merged_congestion.head()

small_section_idx    tti  speed_x  day_of_week        date  hour  speed_y  \
0   88417281d1fffff  1.362    89.30          4.0  2017-06-01     0        7   
1   88417288bdfffff  1.283    42.62          4.0  2017-06-01     0        7   
2   8841728ee5fffff  1.125    92.70          4.0  2017-06-01     0        7   
3   8841728749fffff  1.352    90.60          4.0  2017-06-01     0        7   
4   8841728d87fffff  1.330    54.50          4.0  2017-06-01     0        7   

   precipitation  tempurature  
0            0.1           29  
1            0.1           29  
2            0.1           29  
3            0.1           29  
4            0.1           29

In [82]:
len(merged_congestion)

372976

In [83]:
merged_congestion.to_csv("merged_congestion_nodummy.csv", index = False)

### adding the sectional, day_of_weak, hour dummy

In [84]:
import pandas as pd
merged_congestion = pd.read_csv("merged_congestion_nodummy.csv")

In [85]:
merged_congestion.head()

small_section_idx    tti  speed_x  day_of_week        date  hour  speed_y  \
0   88417281d1fffff  1.362    89.30          4.0  2017-06-01     0        7   
1   88417288bdfffff  1.283    42.62          4.0  2017-06-01     0        7   
2   8841728ee5fffff  1.125    92.70          4.0  2017-06-01     0        7   
3   8841728749fffff  1.352    90.60          4.0  2017-06-01     0        7   
4   8841728d87fffff  1.330    54.50          4.0  2017-06-01     0        7   

   precipitation  tempurature  
0            0.1           29  
1            0.1           29  
2            0.1           29  
3            0.1           29  
4            0.1           29

In [87]:
merged_congestion = merged_congestion.drop(columns = ["speed_x"])

In [89]:
merged_congestion = merged_congestion.rename(columns = {"speed_y":"speed"})

In [90]:
merged_congestion.head()

small_section_idx    tti  day_of_week        date  hour  speed  \
0   88417281d1fffff  1.362          4.0  2017-06-01     0      7   
1   88417288bdfffff  1.283          4.0  2017-06-01     0      7   
2   8841728ee5fffff  1.125          4.0  2017-06-01     0      7   
3   8841728749fffff  1.352          4.0  2017-06-01     0      7   
4   8841728d87fffff  1.330          4.0  2017-06-01     0      7   

   precipitation  tempurature  
0            0.1           29  
1            0.1           29  
2            0.1           29  
3            0.1           29  
4            0.1           29

61.090850830078125

In [91]:
merged_congestion_filtered = merged_congestion.drop(columns = ["date"])

In [92]:
merged_congestion_filtered = merged_congestion_filtered.dropna()

In [93]:
merged_congestion_filtered.head()

small_section_idx    tti  day_of_week  hour  speed  precipitation  \
0   88417281d1fffff  1.362          4.0     0      7            0.1   
1   88417288bdfffff  1.283          4.0     0      7            0.1   
2   8841728ee5fffff  1.125          4.0     0      7            0.1   
3   8841728749fffff  1.352          4.0     0      7            0.1   
4   8841728d87fffff  1.330          4.0     0      7            0.1   

   tempurature  
0           29  
1           29  
2           29  
3           29  
4           29

In [94]:
len(merged_congestion_filtered)

372973

In [96]:

DOW = merged_congestion_filtered["day_of_week"]
day_of_weak = [int(i) for i in DOW]
merged_congestion_filtered["day_of_week"] = day_of_weak

In [97]:
merged_congestion_filtered.head()

small_section_idx    tti  day_of_week  hour  speed  precipitation  \
0   88417281d1fffff  1.362            4     0      7            0.1   
1   88417288bdfffff  1.283            4     0      7            0.1   
2   8841728ee5fffff  1.125            4     0      7            0.1   
3   8841728749fffff  1.352            4     0      7            0.1   
4   8841728d87fffff  1.330            4     0      7            0.1   

   tempurature  
0           29  
1           29  
2           29  
3           29  
4           29

In [98]:
merged_congestion_filtered = pd.get_dummies(merged_congestion_filtered, columns = ["hour"])


In [104]:
merged_congestion_filtered.head()

tti  speed  precipitation  tempurature  hour_0  hour_1  hour_2  hour_3  \
0  1.362      7            0.1           29       1       0       0       0   
1  1.283      7            0.1           29       1       0       0       0   
2  1.125      7            0.1           29       1       0       0       0   
3  1.352      7            0.1           29       1       0       0       0   
4  1.330      7            0.1           29       1       0       0       0   

   hour_4  hour_5                ...                  \
0       0       0                ...                   
1       0       0                ...                   
2       0       0                ...                   
3       0       0                ...                   
4       0       0                ...                   

   small_section_idx_88417295b1fffff  small_section_idx_88417295c1fffff  \
0                                  0                                  0   
1                                  0                                  0   
2                                  0                                  0   
3                                  0                                  0   
4                                  0                                  0   

   small_section_idx_88417295c3fffff  small_section_idx_88417295c7fffff  \
0                                  0                                  0   
1                                  0                                  0   
2                                  0                                  0   
3                                  0                                  0   
4                                  0                                  0   

   small_section_idx_88417295cbfffff  small_section_idx_88417295cdfffff  \
0                                  0                                  0   
1                                  0                                  0   
2                                  0                                  0   
3                                  0                                  0   
4                                  0                                  0   

   small_section_idx_88417295d1fffff  small_section_idx_88417295e1fffff  \
0                                  0                                  0   
1                                  0                                  0   
2                                  0                                  0   
3                                  0                                  0   
4                                  0                                  0   

   small_section_idx_88417295e3fffff  small_section_idx_88417295e7fffff  
0                                  0                                  0  
1                                  0                                  0  
2                                  0                                  0  
3                                  0                                  0  
4                                  0                                  0  

[5 rows x 576 columns]

In [105]:
merged_congestion_filtered.to_csv("merged_congestion_dummy.csv",index = False)